## Chess Tactics Analysis

Using requests and selenium to scrape data from my Chess.com and ChessTempo tactics records.

In [1]:
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import csv
import datetime
import time

In [2]:
# use requests to get html source (Chess.com) with decorated URL
data = requests.get("https://www.chess.com/stats/puzzles/deadlyknightx").text

In [3]:
# scrape recent activity
recent = pd.read_html(data, attrs={'class':'table progress-table problems-table with-row-highlight'})[0]

In [4]:
# scrape tactic themes
motifs = pd.read_html(data, attrs={'class':'progress-table'})[0]

In [5]:
import os  
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options 

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

In [6]:
# use selenium to login to chesstempo.com
driver = webdriver.Chrome("chromedriver.exe", options=options)
driver.get("https://chesstempo.com/chess-statistics.html")
driver.find_element_by_id("usernameField").send_keys("USERNAME") # send username
driver.find_element_by_id("passwordField").send_keys("PASSWORD") # send password
driver.find_element_by_id("loginButton").click()
time.sleep(10)

# scrape table
page_1 = pd.read_html(driver.page_source)[2]

# click next
time.sleep(5)
driver.find_element_by_id("yui-pg0-0-next-link").click()
time.sleep(5)

# scrape table
page_2 = pd.read_html(driver.page_source)[2]

# close browser
driver.close()

In [7]:
# combine data from both pages
ctempo = page_1.append(page_2, ignore_index=True)

### Data Transformation - Chess.com

In [8]:
recent.head()

,Date,ID,Rating,Moves,Target Time,My Time,Avg Time,Outcome,My Rating
0,"Sep 7, 2019",742186,1616,3/3,0:36,1:49,0:34,(80% | +7),1627
1,"Sep 7, 2019",768668,1490,3/3,0:31,13:59,0:58,(70% | +3),1620
2,"Sep 7, 2019",497058,1761,1/4,0:51,1:37,1:10,(23% | -5),1617
3,"Sep 7, 2019",606100,1923,5/5,1:08,2:35,1:33,(87% | +14),1622
4,"Sep 7, 2019",703298,1326,3/3,0:26,3:06,0:53,(70% | +2),1608


In [9]:
# replace the (  |  ) in the Outcome column
recent['Outcome'] = recent['Outcome'].str.replace('(','').str.replace(')','').str.replace('|','')

In [10]:
# split columns - create white player, black player, white rating, black rating
new = recent.Outcome.str.split(" ", n=2, expand=True)

In [11]:
# add back to dataframe
recent['Outcome'] = new[2]
recent['Outcome %'] = new[0].str.replace('%','').astype(int)

In [12]:
recent.head()

,Date,ID,Rating,Moves,Target Time,My Time,Avg Time,Outcome,My Rating,Outcome %
0,"Sep 7, 2019",742186,1616,3/3,0:36,1:49,0:34,+7,1627,80
1,"Sep 7, 2019",768668,1490,3/3,0:31,13:59,0:58,+3,1620,70
2,"Sep 7, 2019",497058,1761,1/4,0:51,1:37,1:10,-5,1617,23
3,"Sep 7, 2019",606100,1923,5/5,1:08,2:35,1:33,+14,1622,87
4,"Sep 7, 2019",703298,1326,3/3,0:26,3:06,0:53,+2,1608,70


In [13]:
recent['Outcome'] = recent['Outcome'].str.replace(' ', '')

In [14]:
motifs.head()

,Category,Score,Unnamed: 2
0,Bishop Pair,90%,Bishop Pair Attempts: 3 Passed: 3 Failed: 0...
1,Knight Outpost,79%,Knight Outpost Attempts: 2 Passed: 2 Failed...
2,Windmill,76%,Windmill Attempts: 6 Passed: 5 Failed: 1 A...
3,Underpromotion,76%,Underpromotion Attempts: 3 Passed: 2 Failed...
4,Two Rooks Checkmate,70%,Two Rooks Checkmate Attempts: 1 Passed: 1 F...


In [15]:
# split into attempts, passed, failed, avg score
new = motifs['Unnamed: 2'].str.split('Attempts: ', expand=True)
new = new[1].str.split('Passed: ', expand=True)
motifs['Attempts'] = new[0]

In [16]:
new = new[1].str.split('Failed: ', expand=True)
motifs['Passed'] = new[0]

new = new[1].str.split('Avg Score: ', expand=True)
motifs['Failed'] = new[0]

In [17]:
motifs['Avg Score %'] = new[1]

In [18]:
motifs.head()

,Category,Score,Unnamed: 2,Attempts,Passed,Failed,Avg Score %
0,Bishop Pair,90%,Bishop Pair Attempts: 3 Passed: 3 Failed: 0...,3,3,0,90.00
1,Knight Outpost,79%,Knight Outpost Attempts: 2 Passed: 2 Failed...,2,2,0,78.50
2,Windmill,76%,Windmill Attempts: 6 Passed: 5 Failed: 1 A...,6,5,1,76.17
3,Underpromotion,76%,Underpromotion Attempts: 3 Passed: 2 Failed...,3,2,1,75.67
4,Two Rooks Checkmate,70%,Two Rooks Checkmate Attempts: 1 Passed: 1 F...,1,1,0,70.00


In [19]:
motifs = motifs.drop(['Unnamed: 2'], axis=1)

### Data Transformation - ChessTempo

In [20]:
ctempo['User Rating'] = ctempo['User Rating'].str.replace('(','').str.replace(')','')

In [21]:
new = ctempo['User Rating'].str.split(' ', expand=True)
ctempo['User Rating'] = new[0]
ctempo['Outcome'] = new[1]

### Add Calculated Columns

In [22]:
# add pass or fail to recent
pf = np.array([])

for i in recent['Outcome']:
    if "+" in i:
        pf = np.append(pf, "Pass")
    else:
        pf = np.append(pf, "Fail")

In [23]:
recent['P/F'] = pf

In [24]:
recent.head()

,Date,ID,Rating,Moves,Target Time,My Time,Avg Time,Outcome,My Rating,Outcome %,P/F
0,"Sep 7, 2019",742186,1616,3/3,0:36,1:49,0:34,+7,1627,80,Pass
1,"Sep 7, 2019",768668,1490,3/3,0:31,13:59,0:58,+3,1620,70,Pass
2,"Sep 7, 2019",497058,1761,1/4,0:51,1:37,1:10,-5,1617,23,Fail
3,"Sep 7, 2019",606100,1923,5/5,1:08,2:35,1:33,+14,1622,87,Pass
4,"Sep 7, 2019",703298,1326,3/3,0:26,3:06,0:53,+2,1608,70,Pass


In [25]:
# add pass or fail to ctempo
pf = np.array([])

for i in ctempo['Outcome']:
    if '+' in i:
        pf = np.append(pf, "Pass")
    else:
        pf = np.append(pf, "Fail")

In [26]:
ctempo['P/F'] = pf

In [27]:
# add chess.com problem URLS
urls = np.array([])

for i in recent['ID']:
    urls = np.append(urls, "https://www.chess.com/puzzles/problem/" + str(i))

In [28]:
recent['URL'] = urls

In [29]:
# add chess tempo problem URLS
ct_urls = np.array([])

for i in ctempo['ProblemId']:
    ct_urls = np.append(ct_urls, "https://chesstempo.com/chess-problems/" + str(i))

In [30]:
ctempo['URL'] = ct_urls

In [35]:
recent.head()

,Date,ID,Rating,Moves,Target Time,My Time,Avg Time,Outcome,My Rating,Outcome %,P/F,URL
0,"Sep 7, 2019",742186,1616,3/3,0:36,1:49,0:34,+7,1627,80,Pass,https://www.chess.com/puzzles/problem/742186
1,"Sep 7, 2019",768668,1490,3/3,0:31,13:59,0:58,+3,1620,70,Pass,https://www.chess.com/puzzles/problem/768668
2,"Sep 7, 2019",497058,1761,1/4,0:51,1:37,1:10,-5,1617,23,Fail,https://www.chess.com/puzzles/problem/497058
3,"Sep 7, 2019",606100,1923,5/5,1:08,2:35,1:33,+14,1622,87,Pass,https://www.chess.com/puzzles/problem/606100
4,"Sep 7, 2019",703298,1326,3/3,0:26,3:06,0:53,+2,1608,70,Pass,https://www.chess.com/puzzles/problem/703298


In [32]:
ctempo.head()

,#,Time,ProblemId,Rating,Type,Av Time,Solve Time,After First,User Rating,Outcome,P/F,URL
0,1,2019-09-07 11:23:28,53947,1374.2,Standard,02:31,03:38,01:19,1569.5,+1.8,Pass,https://chesstempo.com/chess-problems/53947
1,2,2019-09-07 11:19:46,92238,1417.5,Standard,01:53,00:45,00:03,1567.6,+2.3,Pass,https://chesstempo.com/chess-problems/92238
2,3,2019-09-07 11:18:57,173690,1296.2,Standard,01:33,00:37,00:02,1565.3,+1.4,Pass,https://chesstempo.com/chess-problems/173690
3,4,2019-09-07 11:17:48,23857,1599.4,Standard,03:32,01:18,00:00,1563.9,-3.7,Fail,https://chesstempo.com/chess-problems/23857
4,5,2019-09-07 11:16:14,45302,1431.7,Standard,02:03,04:25,00:04,1567.6,+2.7,Pass,https://chesstempo.com/chess-problems/45302


### Export to CSV

In [33]:
# export to CSV file - to be used as Power BI data model
ctempo.to_csv(r"C:\Users\lrspe\Dropbox\Data Science\Chess\chessTempo.csv")
recent.to_csv(r"C:\Users\lrspe\Dropbox\Data Science\Chess\chessTactics.csv")
motifs.to_csv(r"C:\Users\lrspe\Dropbox\Data Science\Chess\chessMotifs.csv")